In [48]:
from sklearn.feature_extraction.text import CountVectorizer
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
from transformers import BertTokenizer, TFBertModel
import tensorflow as tf
from sklearn.cluster import KMeans
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np

In [49]:
kaggle = False

In [50]:
filepath = 'data/reviews.csv' if not kaggle else './kaggle/input/consumer-review-of-clothing-product/Consumer Review of Clothing Product/data_amazon.xlsx - Sheet1.csv'

df = pd.read_csv(filepath)

display(df)

,Title,Review,Cons_rating,Cloth_class,Materials,Construction,Color,Finishing,Durability
0,NaN,Absolutely wonderful - silky and sexy and comf...,4.0,Intimates,0.0,0.0,0.0,1.0,0.0
1,NaN,Love this dress! it's sooo pretty. i happene...,5.0,Dresses,0.0,1.0,0.0,0.0,0.0
2,Some major design flaws,I had such high hopes for this dress and reall...,3.0,Dresses,0.0,0.0,0.0,1.0,0.0
3,My favorite buy!,"I love, love, love this jumpsuit. it's fun, fl...",5.0,Pants,0.0,0.0,0.0,0.0,0.0
4,Flattering shirt,This shirt is very flattering to all due to th...,5.0,Blouses,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
49333,Dress felt and fit great. I got lots of compl...,Loved the color!!! Dress fit great and I got ...,5.0,Dresses,0.0,0.0,1.0,0.0,0.0
49334,Loved the dress but poor quality,This dress looked great and I loved the materi...,2.0,Dresses,1.0,0.0,0.0,0.0,1.0
49335,"Cute dress, didn't fit",Wanted this dress to work it didn't. It is ver...,1.0,Dresses,0.0,1.0,0.0,0.0,0.0
49336,Very cute!,No complaints othe than the zipper gets stuck ...,4.0,Dresses,0.0,0.0,0.0,0.0,1.0


In [51]:
clean_df = df.dropna(subset = ['Title', 'Review']).drop(columns = ['Materials', 'Construction', 'Color', 'Finishing', 'Durability'])
clean_df.head()

,Title,Review,Cons_rating,Cloth_class
2,Some major design flaws,I had such high hopes for this dress and reall...,3.0,Dresses
3,My favorite buy!,"I love, love, love this jumpsuit. it's fun, fl...",5.0,Pants
4,Flattering shirt,This shirt is very flattering to all due to th...,5.0,Blouses
5,Not for the very petite,"I love tracy reese dresses, but this one is no...",2.0,Dresses
6,Cagrcoal shimmer fun,I aded this in my basket at hte last mintue to...,5.0,Knits


In [52]:
text_features = clean_df[['Review', 'Cloth_class','Title']].copy()
text_features

,Review,Cloth_class,Title
2,I had such high hopes for this dress and reall...,Dresses,Some major design flaws
3,"I love, love, love this jumpsuit. it's fun, fl...",Pants,My favorite buy!
4,This shirt is very flattering to all due to th...,Blouses,Flattering shirt
5,"I love tracy reese dresses, but this one is no...",Dresses,Not for the very petite
6,I aded this in my basket at hte last mintue to...,Knits,Cagrcoal shimmer fun
...,...,...,...
49333,Loved the color!!! Dress fit great and I got ...,Dresses,Dress felt and fit great. I got lots of compl...
49334,This dress looked great and I loved the materi...,Dresses,Loved the dress but poor quality
49335,Wanted this dress to work it didn't. It is ver...,Dresses,"Cute dress, didn't fit"
49336,No complaints othe than the zipper gets stuck ...,Dresses,Very cute!


In [53]:
def tfidf_vectorizer(text):
    vectorizer = CountVectorizer()
    X = vectorizer.fit_transform(text)
    return X



In [54]:
test_tfidf = tfidf_vectorizer(text_features['Review'])

In [55]:
#test_word2vec = word2vec_vectorizer(text_features['Review'])

In [56]:
#test_bert = bert_vectorizer(text_features['Review'])

In [57]:
test_tfidf

<45321x19792 sparse matrix of type '<class 'numpy.int64'>'
	with 1397815 stored elements in Compressed Sparse Row format>